In [1]:
import unicodedata
import re
import os
import json
import requests
from bs4 import BeautifulSoup 

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd
import numpy as np

import acquire as a

In [2]:
##### Imports #####
import unicodedata
import re
import os
import json
import requests
from bs4 import BeautifulSoup 

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd
import numpy as np

import acquire as a


##### Funcitons #####

def basic_clean(string):
    article = string.lower()
    article = unicodedata.normalize('NFKD', article).encode('ascii', 'ignore').decode('utf-8')
    #article = re.sub(r"[^\w]", ' ', article)
    article = re.sub("[^a-z0-9'\s]", '', article)
    article = re.sub("\\n", ' ', article)

    return article


def tokenize(string):
    article = []
    # Create the tokenizer
    tokenizer = nltk.tokenize.ToktokTokenizer()

    # Use the tokenizer
    article = tokenizer.tokenize(string, return_str = True)

    return article


def stem(string):
    ps = nltk.porter.PorterStemmer()
    stems = [ps.stem(word) for word in string.split()]
    article_stemmed = ' '.join(stems)
    
    return article_stemmed


def lemmatize(text):
    # Create the Lemmatizer.
    wnl = nltk.stem.WordNetLemmatizer()
    # use the lemmatizer
    lemmas = [wnl.lemmatize(word) for word in text.split()]
    # Join the lemmatized words together
    article_lemmatized = ' '.join(lemmas)
    
    return article_lemmatized


def remove_stopwords(string, extra_words=['build', 'file', 'example', 'code', 'use'], exclude_words=[]):   
    stopword_list = stopwords.words('english')
    # Remove 'exclude_words' from stopword_list to keep these in my text.
    stopword_list = set(stopword_list) - set(exclude_words)
    # Add in 'extra_words' to stopword_list.
    stopword_list = stopword_list.union(set(extra_words))
    # Split words in lemmatized article.
    words = string.split()
    # Iterate through the words and only keep those not in stopword_list
    filtered_words = [word for word in words if word not in stopword_list]
    # Join words in the list back into strings; assign to a variable to keep changes.
    article_without_stopwords = ' '.join(filtered_words)

    return article_without_stopwords

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jasontellez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jasontellez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
df = pd.read_json('data.json')
df

,repo,language,readme_contents
0,microsoft/react-native-windows,C++,"<h1 align=""center""> React Native for Windows <..."
1,microsoft/fast,TypeScript,\n![fast_banner_github_914.png](https://static...
2,microsoft/Application-Insights-Workbooks,JSON,# Azure Monitor Workbook Templates [![Build St...
3,microsoft/gctoolkit,Java,# Microsoft GCToolKit\n\nGCToolkit is a set of...
4,microsoft/winget-cli-restsource,C#,# Welcome to the winget-cli-restsource reposit...
...,...,...,...
1495,microsoft/AzureClusterlessHPC.jl,Julia,[![](https://img.shields.io/badge/docs-stable-...
1496,microsoft/multicluster-gitops,Shell,# Multi-cluster and multi-tenant environment w...
1497,microsoft/kfp-event-handler,Go,# Kubeflow Pipelines Event Handler\n\nThis mic...
1498,microsoft/Build-Docker-Provider,Shell,# Build-Docker-Provider\nSuperproject for Dock...


In [5]:
df['clean'] = df['readme_contents'].apply(basic_clean)\
.apply(tokenize)\
.apply(lemmatize)\
.apply(remove_stopwords)
df

,repo,language,readme_contents,clean
0,microsoft/react-native-windows,C++,"<h1 align=""center""> React Native for Windows <...",h1 aligncenter react native windows h1 p align...
1,microsoft/fast,TypeScript,\n![fast_banner_github_914.png](https://static...,fastbannergithub914pnghttpsstaticfastdesignass...
2,microsoft/Application-Insights-Workbooks,JSON,# Azure Monitor Workbook Templates [![Build St...,azure monitor workbook templates statushttpsgi...
3,microsoft/gctoolkit,Java,# Microsoft GCToolKit\n\nGCToolkit is a set of...,microsoft gctoolkit gctoolkit set libraries an...
4,microsoft/winget-cli-restsource,C#,# Welcome to the winget-cli-restsource reposit...,welcome wingetclirestsource repository buildin...
...,...,...,...,...
1495,microsoft/AzureClusterlessHPC.jl,Julia,[![](https://img.shields.io/badge/docs-stable-...,httpsimgshieldsiobadgedocsstablebluesvghttpsmi...
1496,microsoft/multicluster-gitops,Shell,# Multi-cluster and multi-tenant environment w...,multicluster multitenant environment flux v2 r...
1497,microsoft/kfp-event-handler,Go,# Kubeflow Pipelines Event Handler\n\nThis mic...,kubeflow pipelines event handler microservice ...
1498,microsoft/Build-Docker-Provider,Shell,# Build-Docker-Provider\nSuperproject for Dock...,builddockerprovider superproject dockerprovide...


In [6]:
df.clean = df.clean.str.strip()

In [7]:
df

,repo,language,readme_contents,clean
0,microsoft/react-native-windows,C++,"<h1 align=""center""> React Native for Windows <...",h1 aligncenter react native windows h1 p align...
1,microsoft/fast,TypeScript,\n![fast_banner_github_914.png](https://static...,fastbannergithub914pnghttpsstaticfastdesignass...
2,microsoft/Application-Insights-Workbooks,JSON,# Azure Monitor Workbook Templates [![Build St...,azure monitor workbook templates statushttpsgi...
3,microsoft/gctoolkit,Java,# Microsoft GCToolKit\n\nGCToolkit is a set of...,microsoft gctoolkit gctoolkit set libraries an...
4,microsoft/winget-cli-restsource,C#,# Welcome to the winget-cli-restsource reposit...,welcome wingetclirestsource repository buildin...
...,...,...,...,...
1495,microsoft/AzureClusterlessHPC.jl,Julia,[![](https://img.shields.io/badge/docs-stable-...,httpsimgshieldsiobadgedocsstablebluesvghttpsmi...
1496,microsoft/multicluster-gitops,Shell,# Multi-cluster and multi-tenant environment w...,multicluster multitenant environment flux v2 r...
1497,microsoft/kfp-event-handler,Go,# Kubeflow Pipelines Event Handler\n\nThis mic...,kubeflow pipelines event handler microservice ...
1498,microsoft/Build-Docker-Provider,Shell,# Build-Docker-Provider\nSuperproject for Dock...,builddockerprovider superproject dockerprovide...


In [8]:
df['lemmatized'] = df.clean.apply(lemmatize)

In [9]:
df

,repo,language,readme_contents,clean,lemmatized
0,microsoft/react-native-windows,C++,"<h1 align=""center""> React Native for Windows <...",h1 aligncenter react native windows h1 p align...,h1 aligncenter react native window h1 p alignc...
1,microsoft/fast,TypeScript,\n![fast_banner_github_914.png](https://static...,fastbannergithub914pnghttpsstaticfastdesignass...,fastbannergithub914pnghttpsstaticfastdesignass...
2,microsoft/Application-Insights-Workbooks,JSON,# Azure Monitor Workbook Templates [![Build St...,azure monitor workbook templates statushttpsgi...,azure monitor workbook template statushttpsgit...
3,microsoft/gctoolkit,Java,# Microsoft GCToolKit\n\nGCToolkit is a set of...,microsoft gctoolkit gctoolkit set libraries an...,microsoft gctoolkit gctoolkit set library anal...
4,microsoft/winget-cli-restsource,C#,# Welcome to the winget-cli-restsource reposit...,welcome wingetclirestsource repository buildin...,welcome wingetclirestsource repository buildin...
...,...,...,...,...,...
1495,microsoft/AzureClusterlessHPC.jl,Julia,[![](https://img.shields.io/badge/docs-stable-...,httpsimgshieldsiobadgedocsstablebluesvghttpsmi...,httpsimgshieldsiobadgedocsstablebluesvghttpsmi...
1496,microsoft/multicluster-gitops,Shell,# Multi-cluster and multi-tenant environment w...,multicluster multitenant environment flux v2 r...,multicluster multitenant environment flux v2 r...
1497,microsoft/kfp-event-handler,Go,# Kubeflow Pipelines Event Handler\n\nThis mic...,kubeflow pipelines event handler microservice ...,kubeflow pipeline event handler microservice e...
1498,microsoft/Build-Docker-Provider,Shell,# Build-Docker-Provider\nSuperproject for Dock...,builddockerprovider superproject dockerprovide...,builddockerprovider superproject dockerprovide...


In [10]:
df.dropna(subset=['language'], inplace=True)

In [11]:
len(df)

1376

In [12]:
df.loc[0].lemmatized

"h1 aligncenter react native window h1 p aligncenter native window apps react p p aligncenter hrefhttpsgithubcommicrosoftreactnativewindowsblobmainlicense img srchttpsimgshieldsiobadgelicensemitbluesvg altreact native window released mit license hrefhttpswwwnpmjsorgpackagereactnativewindows img srchttpsimgshieldsionpmvreactnativewindowscolore80441labelreactnativewindows altcurrent npm package version hrefhttpsgithubcommicrosoftreactnativewindowscontributing img srchttpsimgshieldsiobadgeprswelcomebrightgreensvg altprs welcome p hero image logohttpsgithubcommicrosoftreactnativewindowsrawmaingithubhero2png see official react native websitehttpsreactnativedev introduction react native react nativehttpsreactnativedev framework developed facebook enables worldclass application experience native platform using consistent developer experience based javascript reacthttpsreactjsorg focus react native developer efficiency across platform care learn write anywhere repository add support window 10 

In [13]:
df = df[['repo', 'language', 'lemmatized']]

In [14]:
df = df.rename(columns={"lemmatized": "content"})

In [15]:
df = df.reset_index(drop=True)

In [16]:
df.to_csv('cleaned_readmes.csv')

In [17]:
stopword_list = stopwords.words('english')
stopword_list

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [19]:
df

,repo,language,content
0,microsoft/react-native-windows,C++,h1 aligncenter react native window h1 p alignc...
1,microsoft/fast,TypeScript,fastbannergithub914pnghttpsstaticfastdesignass...
2,microsoft/Application-Insights-Workbooks,JSON,azure monitor workbook template statushttpsgit...
3,microsoft/gctoolkit,Java,microsoft gctoolkit gctoolkit set library anal...
4,microsoft/winget-cli-restsource,C#,welcome wingetclirestsource repository buildin...
...,...,...,...
1371,microsoft/AzureClusterlessHPC.jl,Julia,httpsimgshieldsiobadgedocsstablebluesvghttpsmi...
1372,microsoft/multicluster-gitops,Shell,multicluster multitenant environment flux v2 r...
1373,microsoft/kfp-event-handler,Go,kubeflow pipeline event handler microservice e...
1374,microsoft/Build-Docker-Provider,Shell,builddockerprovider superproject dockerprovide...


In [24]:
pd.Series(' '.join(df.content.astype('str')).split()).isin(pd.Series('using')).mean()

0.005495384261065938